### 전이학습


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

from torchinfo import summary
from torchmetrics.functional.classification import multiclass_accuracy


In [15]:
### ===> 데이터 준비
img_dir = "../data/img"

### ===> Resnet 전처리
# resize_size=[256] interpolation=InterpolationMode.BILINEAR
# crop_size=[224]
# first rescaled to [0.0, 1.0] and then
# normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225]
preprocessing = transforms.Compose(
    [
        transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.299, 0.224, 0.225)),
    ]
)

### ===> 이미지 데이터셋 생성
imgDS = ImageFolder(img_dir, transform=preprocessing)

print(imgDS.classes, imgDS.targets, imgDS.imgs, sep="\n")


['cat', 'dysonsphere', 'flower']
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2]
[('../data/img\\cat\\cat.jpg', 0), ('../data/img\\cat\\cat01.jpg', 0), ('../data/img\\cat\\cat02.jpg', 0), ('../data/img\\cat\\cat03.jpg', 0), ('../data/img\\cat\\cat04.jpg', 0), ('../data/img\\cat\\cat05.jpg', 0), ('../data/img\\dysonsphere\\DysonSphere.jpg', 1), ('../data/img\\dysonsphere\\DysonSphere01.jpg', 1), ('../data/img\\dysonsphere\\DysonSphere02.jpg', 1), ('../data/img\\dysonsphere\\DysonSphere03.jpg', 1), ('../data/img\\dysonsphere\\DysonSphere04.jpg', 1), ('../data/img\\dysonsphere\\DysonSphere05.jpg', 1), ('../data/img\\flower\\flower01.jpg', 2), ('../data/img\\flower\\flower02.jpg', 2), ('../data/img\\flower\\flower03.jpg', 2)]


In [16]:
### ===> 데이터 로더 생성
imgDL = DataLoader(imgDS, batch_size=3, shuffle=True, drop_last=True)
for img, label in imgDL:
    print(img.shape, label)


torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 2, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 2, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 2, 1])


In [17]:
### ==> 모델 설계 및 설정

### 사전학습된 모델 인스턴스 생성
res_model = resnet18(weights=ResNet18_Weights.DEFAULT)

### 전결합층 변경
### in_feature : FeatureMap에서 받은 피쳐 수, out_features : 출력/분류 클래스 수
res_model.fc = nn.Linear(in_features=512, out_features=len(imgDS.classes))


In [18]:
summary(res_model, (3, 3, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [3, 3]                    --
├─Conv2d: 1-1                            [3, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [3, 64, 112, 112]         128
├─ReLU: 1-3                              [3, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [3, 64, 56, 56]           --
├─Sequential: 1-5                        [3, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [3, 64, 56, 56]           --
│

In [19]:
for idx, module in enumerate(res_model.modules()):
    print(f"[{idx}]==========\n", module)


[0]==========
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(i

In [20]:
### Resnet18 Feature Module 파라미터 require_grad=True ==> False 비활성화
for name, param in res_model.named_parameters():
    print(name, param.requires_grad, end="  ===>    ")
    param.requires_grad = False
    print(param.requires_grad)

### Resnet18 Full Connected Module 파라미터 require_grad=False ==> True 활성화
for name, param in res_model.fc.named_parameters():
    print(name, param.requires_grad, end="  ===>    ")
    param.requires_grad = True
    print(param.requires_grad)


conv1.weight True  ===>    False
bn1.weight True  ===>    False
bn1.bias True  ===>    False
layer1.0.conv1.weight True  ===>    False
layer1.0.bn1.weight True  ===>    False
layer1.0.bn1.bias True  ===>    False
layer1.0.conv2.weight True  ===>    False
layer1.0.bn2.weight True  ===>    False
layer1.0.bn2.bias True  ===>    False
layer1.1.conv1.weight True  ===>    False
layer1.1.bn1.weight True  ===>    False
layer1.1.bn1.bias True  ===>    False
layer1.1.conv2.weight True  ===>    False
layer1.1.bn2.weight True  ===>    False
layer1.1.bn2.bias True  ===>    False
layer2.0.conv1.weight True  ===>    False
layer2.0.bn1.weight True  ===>    False
layer2.0.bn1.bias True  ===>    False
layer2.0.conv2.weight True  ===>    False
layer2.0.bn2.weight True  ===>    False
layer2.0.bn2.bias True  ===>    False
layer2.0.downsample.0.weight True  ===>    False
layer2.0.downsample.1.weight True  ===>    False
layer2.0.downsample.1.bias True  ===>    False
layer2.1.conv1.weight True  ===>    False


In [21]:
### ===> 학습 준비
optimizer = optim.Adam(res_model.parameters())
loss_fn = nn.CrossEntropyLoss()  # 손실함수 정의
EPOCHS = 3
